In [1]:
import pandas as pd
df = pd.read_csv('sistema-unico-de-atencion-ciudadana-2021.csv', delimiter=';',low_memory=False)
df['prestacion'] = df['prestacion'].str.upper()

In [2]:
prestaciones_residuos = [
    'LIMPIEZA DE CAMPANA Y/O RESIDUOS DISEMINADOS ALREDEDOR', 
    'INSTALACIÓN DE CAMPANA VERDE', 
    'REPARACIÓN DE CAMPANA VERDE', 
    'VACIADO DE CAMPANA VERDES', 
    'RESIDUOS ACUMULADOS O DISEMINADOS FUERA DE PUNTO VERDE', 
    'RECUPERADOR URBANO ACOPIANDO RESIDUOS EN ZONA NO PREVISTA', 
    'ASENTAMIENTO DE RECUPERADORES URBANOS EN ESPACIO PÚBLICO', 
    'USO INDEBIDO DEL AGUA PARA LIMPIAR LAS VEREDAS', 
    'USO INDEBIDO DEL PLÁSTICO (BOLSAS NO REUTILIZABLES Y SORBETES)', 
    'IRREGULARIDADES EN LA SEPARACIÓN DE RESIDUOS EN ORIGEN', 
    'RECOLECCIÓN DE RESIDUOS SECOS EN GRAN GENERADOR', 
    'RECOLECCIÓN DE RESIDUOS FUERA DEL CONTENEDOR', 
    'RETIRO DE RESIDUOS VOLUMINOSOS (MUEBLES Y ELECTRODOMÉSTICOS)', 
    'IRREGULARIDADES EN LA MANIPULACIÓN DE RESIDUOS PELIGROSOS', 
    'IRREGULARIDADES EN LA DISPOSICIÓN DE RESIDUOS DE ESTABLECIMIENTOS', 
    'PERSONAL DE EMPRESA DE RECOLECCIÓN CON CONDUCTAS INDEBIDAS'] 
prestaciones_accecibilidad = [
    'CONSTRUCCIÓN DE RAMPA DE ACCESIBILIDAD',
    'REPARACIÓN DE RAMPA DE ACCESIBILIDAD',
    'FALTA DE RAMPAS DE ACCESO PARA PERSONAS CON DISCAPACIDAD MOTRIZ EN ESTABLECIMIENTOS NO COMERCIALES',
    'FALTA DE RAMPAS DE ACCESO PARA PERSONAS CON DISCAPACIDAD MOTRIZ EN LOCAL COMERCIAL',
    'IRREGULARIDADES CON RESERVAS DE ESPACIOS PARA ESTACIONAMIENTO DE VEHÍCULOS DE PERSONAS CON DISCAPACIDAD',
    'IRREGULARIDADES EN EL USO DEL CERTIFICADO DE DISCAPACIDAD PARA ESTACIONAMIENTO',
    'FALTA DE RAMPAS DE ACCESO PARA PERSONAS CON DISCAPACIDAD MOTRIZ',
    'CONSTRUCCIÓN / REPARACIÓN DE DÁRSENA PARA ASCENSO/DESCENSO DE COLECTIVOS',
    'REPARACIÓN DE VEREDA',
    'OBSTRUCCIÓN DE CALLE O VEREDA POR ELEMENTOS FIJOS O MÓVILES'
]

residuos = df[df['prestacion'].isin(prestaciones_accecibilidad)]
residuos.shape

(23280, 20)

In [ ]:
# formateo de fechas y horas
def date_transformation(df):
    df['mes'] = df['periodo'].astype(str)
    df['mes'] = df['mes'].str[-2:]

    dfs_por_mes = [df[df['mes'] == mes] for mes in df['mes'].unique()]
    
    for i, mes_df in enumerate(dfs_por_mes):
        if mes_df['mes'].iloc[0] in ['01', '02', '03', '04', '05']:
            dfs_por_mes[i]['fecha_ingros_stan'] = pd.to_datetime(mes_df['fecha_ingreso'], dayfirst=True)
        else:
            dfs_por_mes[i]['fecha_ingros_stan'] = pd.to_datetime(mes_df['fecha_ingreso'], dayfirst=False)

    df = pd.concat(dfs_por_mes)

    return df

residuos = date_transformation(residuos)

residuos['fecha_ingreso'] = residuos['fecha_ingros_stan']
residuos['hora_ingreso'] = pd.to_datetime(residuos['hora_ingreso']).dt.time.astype(str)

# residuos['fecha_hora_ingreso'] = pd.to_datetime(residuos['fecha_ingreso'] + ' ' + residuos['hora_ingreso'])
residuos['fecha_hora_ingreso'] = residuos['fecha_ingreso'] + pd.to_timedelta(residuos['hora_ingreso'].astype(str))

residuos.shape

In [ ]:
# formateo de fechas y horas cierre

def date_transformation_2(df):
    df['mes'] = df['periodo'].astype(str)
    df['mes'] = df['mes'].str[-2:]

    dfs_por_mes = [df[df['mes'] == mes] for mes in df['mes'].unique()]
    
    for i, mes_df in enumerate(dfs_por_mes):
        if mes_df['mes'].iloc[0] in ['01', '02', '03', '04', '05']:
            dfs_por_mes[i]['fecha_cierre_stan'] = pd.to_datetime(mes_df['fecha_cierre'], dayfirst=True)
        else:
            dfs_por_mes[i]['fecha_cierre_stan'] = pd.to_datetime(mes_df['fecha_cierre'], dayfirst=False)

    df = pd.concat(dfs_por_mes)

    return df

residuos['fecha_cierre'] = residuos['fecha_cierre_contacto'].astype(str).str[:10]
residuos['hora_cierre'] = residuos['fecha_cierre_contacto'].astype(str).str[11:]

residuos = date_transformation_2(residuos)

residuos['fecha_cierre'] = residuos['fecha_cierre_stan']
residuos['hora_cierre'] = pd.to_datetime(residuos['hora_cierre']).dt.time.astype(str)

residuos['fecha_hora_cierre'] = residuos['fecha_cierre'] + pd.to_timedelta(residuos['hora_cierre'].astype(str))

In [5]:
residuos['domicilio_calle'] = residuos['domiclio_calle'].combine_first(residuos['domicilio_calle'])

# sacar periodo, fecha_ingreso, hora_ingreso, fecha_cierre_contacto, domiclio_calle, mes, fecha_ingreso_stan, fecha_cierre, hora_cierre, fecha_cierre_stan
cols_to_drop = ['fecha_ingreso', 'hora_ingreso', 'fecha_cierre_contacto', 'domiclio_calle', 'mes', 'fecha_ingros_stan', 'fecha_cierre', 'hora_cierre', 'fecha_cierre_stan']
residuos = residuos.drop(cols_to_drop, axis=1)

residuos = residuos[['contacto', 'periodo','categoria', 'subcategoria', 'prestacion', 'tipo_prestacion', 'fecha_hora_ingreso', 'domicilio_comuna', 'domicilio_calle', 'domicilio_altura', 'domicilio_esquina_proxima', 'lat', 'lon', 'canal', 'genero', 'estado_del_contacto', 'fecha_hora_cierre']]
residuos.shape

(23280, 17)

In [7]:
residuos['domicilio_comuna'].fillna(0, inplace=True) # comuna == 0 --> NaN
residuos['domicilio_comuna'] = residuos['domicilio_comuna'].astype(int)
residuos['domicilio_altura'].fillna(0, inplace=True) # altura == 0 --> NaN

residuos['domicilio_altura'] = residuos['domicilio_altura'].astype(int)
residuos['domicilio_calle'] = residuos['domicilio_calle'].str.replace('"', '')

In [8]:
residuos.to_csv('dataset2_accesibilidad.csv', index=False)